In [13]:
import os
from collections import defaultdict
import torch
import numpy as np
import scipy.stats
from torch.distributions import constraints
from matplotlib import pyplot
import random
import time
import pandas as pd

import pyro
import pyro.distributions as dist
from pyro import poutine
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
from pyro.infer import SVI, TraceEnum_ELBO, config_enumerate, infer_discrete, Predictive
from pyro.ops.indexing import Vindex
from pyro.infer import MCMC, NUTS
import torch.nn.functional as F

import pickle

from sklearn.model_selection import KFold

In [16]:
with open('tomtom_data_preprocessed_withadded.pkl','rb') as f:
    [tself_norm_all_3d, tself_norm_noauto_3d, tself_raw_all_3d, tself_raw_noauto_3d,
    ttarg_norm_all_3d, ttarg_norm_noauto_3d, ttarg_raw_all_3d, ttarg_raw_noauto_3d,
    tavg_norm_all_3d, tavg_norm_noauto_3d, tavg_raw_all_3d, tavg_raw_noauto_3d] = pickle.load(f)

kf = KFold(n_splits = 5, shuffle = True, random_state = 888) # make sure these parameters match up with what's used to train/test

testinds = []
for train, test in kf.split(list(range(tself_raw_noauto_3d.shape[0]))):
    testinds.append(test)
    
# print(testinds)


[array([  7,  12,  13,  38,  42,  55,  56,  57,  58,  60,  73,  77,  81,
        83,  87,  91,  95,  97,  98,  99, 101, 103, 114, 117, 119, 123,
       125, 126, 128, 131, 133, 138, 141, 146, 147, 152, 155, 157, 159,
       170, 190, 196, 199, 201, 202, 209, 214, 220, 225, 226, 232, 237,
       238, 245, 247, 257, 264, 271, 275, 277, 291, 293, 294, 298, 301,
       303, 322, 323, 334, 340, 341]), array([  2,   9,  10,  14,  17,  18,  25,  27,  28,  33,  43,  50,  52,
        54,  62,  63,  68,  72,  85,  86, 106, 107, 108, 110, 116, 139,
       145, 160, 164, 165, 166, 174, 175, 178, 184, 185, 187, 191, 192,
       195, 198, 203, 207, 208, 210, 212, 213, 215, 219, 223, 228, 236,
       250, 251, 252, 254, 255, 260, 263, 265, 285, 295, 311, 321, 325,
       330, 332, 336, 337, 345, 350]), array([  0,   1,   4,   5,  15,  19,  20,  24,  30,  32,  34,  36,  40,
        48,  51,  61,  67,  69,  70,  76,  79,  89,  90,  93, 100, 134,
       135, 137, 143, 151, 153, 156, 167, 171, 173, 182, 

In [ ]:
# # read in metrics and put them in a df
# maxk = 10

# allrows = []
# for k in range(maxk):  
#     with open('kfold_metrics_grp_k{}.pkl'.format(k+1),'rb') as f:
#         [ink] = pickle.load(f)
#         # each element in ink is results from a split
#         for split_id in range(len(ink)):
#             true_ids = testinds[split_id]
#             split = ink[split_id]
#             metric_list = [metric for metric in split]
#             [npar,nstimfrom,nstimto] = list(split[0].shape)
#             for participant in range(npar):
#                 for stimfrom in range(nstimfrom):
#                     for stimto in range(nstimto):
#                         pred_list = [metric[participant,stimfrom,stimto] for metric in split]
#                         [npredfrom, npredto] = list(pred_list[0].shape)
#                         for predfrom in range(npredfrom):
#                             for predto in range(npredto):
#                                 single_row = [k+1,true_ids[participant], stimfrom, stimto, predfrom,predto]
#                                 for i in range(len(pred_list)):
#                                     if i == (len(pred_list) - 1): # turns out MixtureSameFamily only returns the total log prob of evertying in the distribution object, so there's no by-item lp for soft (or dimensional)
#                                         single_row.append(pred_list[i].item()/(npredfrom*npredto)) #spread the total logprob across cells to make summing later easier
#                                     else:    
#                                         single_row.append(pred_list[i][predfrom,predto].item())
#                                 allrows.append(np.array(single_row))
#     del ink
                                
# allpred = np.stack(allrows,axis = 0)
# dfgrp = pd.DataFrame(allpred, columns = ['K','pid','stimfrom','stimto','predfrom','predto','h_ae','s_ae','h_se','s_se','h_lp','s_lpX'])
# dfgrp.to_csv('kfold_metrics_grp.csv',index = False)     

# # release memory 
# del allrows
                
        
        
    

In [54]:
allrows = []
maxk = 10
for k in range(maxk):
    for split_id in range(5):       
        with open('kfold_metrics_dim_k{}_split{}.pkl'.format(k+1,split_id+1),'rb') as f:
            [ink] = pickle.load(f)
            true_ids = testinds[split_id]
            split = list(ink)
            split.pop(0)
            metric_list = [metric for metric in split]
            [npar,nstimfrom,nstimto] = list(split[0].shape)
            for participant in range(npar):
                for stimfrom in range(nstimfrom):
                    for stimto in range(nstimto):
                        pred_list = [metric[participant,stimfrom,stimto] for metric in split]
                        [npredfrom, npredto] = list(pred_list[0].shape)
                        for predfrom in range(npredfrom):
                            for predto in range(npredto):
                                single_row = [k+1,true_ids[participant], stimfrom, stimto, predfrom,predto]
                                for i in range(len(pred_list)):
                                    if i == (len(pred_list) - 1): # turns out MixtureSameFamily only returns the total log prob of evertying in the distribution object, so there's no by-item lp for soft (or dimensional)
                                        single_row.append(pred_list[i].item()/(npredfrom*npredto)) #spread the total logprob across cells to make summing later easier
                                    else:    
                                        single_row.append(pred_list[i][predfrom,predto].item())
                                allrows.append(np.array(single_row))
    del ink                           
allpred = np.stack(allrows,axis = 0)
dfgrp = pd.DataFrame(allpred, columns = ['K','pid','stimfrom','stimto','predfrom','predto','d_ae','d_se','d_lpX'])
dfgrp.to_csv('kfold_metrics_dim.csv',index = False) 